<a href="https://colab.research.google.com/github/advik-7/Agents/blob/main/hierarchical_agent_system_For_Customer_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [5]:
from langchain_core.tools import tool
from transformers import pipeline
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI


In [6]:

@tool
def analyze_sentiment(feedback: str) -> str:
    """
    Analyzes the sentiment of customer feedback using a Hugging Face model.
    """
    sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    sentiment = sentiment_pipeline(feedback)[0]
    return f"Sentiment: {sentiment['label']}, Confidence: {sentiment['score']:.2f}"

@tool
def extract_topics(feedback: str) -> str:
    """
    Extracts topics from customer feedback using zero-shot classification.
    """
    topic_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    candidate_labels = ["price", "quality", "customer service", "delivery", "return policy"]
    topics = topic_pipeline(feedback, candidate_labels=candidate_labels)
    top_topic = topics["labels"][0]
    confidence = topics["scores"][0]
    return f"Top Topic: {top_topic}, Confidence: {confidence:.2f}"

@tool
def summarize_feedback(feedback: str) -> str:
    """
    Summarizes customer feedback using a Hugging Face summarization model.
    """
    summarization_pipeline = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarization_pipeline(feedback, max_length=50, min_length=20, do_sample=False)
    return summary[0]['summary_text']


In [7]:
class SubAgent:
    def __init__(self, name, tool):
        self.name = name
        self.tool = tool

    def execute_task(self, input_text: str):
        print(f"{self.name} is processing the input...")
        result = self.tool(input_text)
        print(f"{self.name} result: {result}")
        return result


In [8]:
class MainAgent:
    def __init__(self, sentiment_tool, topic_tool, summarization_tool):
        self.sentiment_agent = SubAgent("Sentiment Analysis Agent", sentiment_tool)
        self.topic_agent = SubAgent("Topic Extraction Agent", topic_tool)
        self.summarization_agent = SubAgent("Summarization Agent", summarization_tool)

    def distribute_tasks(self, feedback):
        print("\nMain Agent is distributing tasks...\n")

        sentiment_result = self.sentiment_agent.execute_task(feedback)

        topic_result = self.topic_agent.execute_task(feedback)

        summary_result = self.summarization_agent.execute_task(feedback)

        print("\n--- Final Results ---")
        print(f"Sentiment: {sentiment_result}")
        print(f"Key Topics: {topic_result}")
        print(f"Summary: {summary_result}")

        return {
            "sentiment": sentiment_result,
            "topics": topic_result,
            "summary": summary_result,
        }


In [10]:
if __name__ == "__main__":
    sentiment_tool = analyze_sentiment
    topic_tool = extract_topics
    summarization_tool = summarize_feedback

    main_agent = MainAgent(sentiment_tool, topic_tool, summarization_tool)

    print("Customer Feedback Analysis\n")
    print("Please enter the customer feedback text below:")
    feedback_text = input("> ")
    print("\nProcessing feedback...")
    results = main_agent.distribute_tasks(feedback_text)

    print("\n--- Summary of Results ---")
    for task, result in results.items():
        print(f"{task.capitalize()}: {result}")


Customer Feedback Analysis

Please enter the customer feedback text below:
>  The shoes are amazing but only issue I was unable to remove stains for the shoes

Processing feedback...

Main Agent is distributing tasks...

Sentiment Analysis Agent is processing the input...
Sentiment Analysis Agent result: Sentiment: NEGATIVE, Confidence: 0.89
Topic Extraction Agent is processing the input...
Topic Extraction Agent result: Top Topic: quality, Confidence: 0.96
Summarization Agent is processing the input...


Your max_length is set to 50, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


Summarization Agent result: The shoes are amazing but only issue I was unable to remove stains for the shoes.

--- Final Results ---
Sentiment: Sentiment: NEGATIVE, Confidence: 0.89
Key Topics: Top Topic: quality, Confidence: 0.96
Summary: The shoes are amazing but only issue I was unable to remove stains for the shoes.

--- Summary of Results ---
Sentiment: Sentiment: NEGATIVE, Confidence: 0.89
Topics: Top Topic: quality, Confidence: 0.96
Summary: The shoes are amazing but only issue I was unable to remove stains for the shoes.
